# 新闻实体观点提取
运行在服务器，standcorenlp 运行比较慢
***

In [167]:
import  pandas as pd
from stanfordcorenlp import StanfordCoreNLP
from gensim.models import KeyedVectors
import random
from pprint import pprint
from pprint import pprint as pp
base_path = '/root/files/'
file_path = base_path + 'news_content.csv'
nlp_path = '/root/stanford-corenlp'
word2vec_model_path = base_path + 'sgns.merge.word.bz2'
end = {'。','!','?'}
say_represents = r

In [3]:
wm = KeyedVectors.load_word2vec_format(word2vec_model_path,
                                       binary=False, 
                                       unicode_errors='ignore')

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 15 µs


In [25]:
from pprint import pprint as pp
def search(query,depth=3):
    r = {}
    all_r = [] 
    topn = 10
    s = wm.most_similar(query,topn=topn)
    r[0] = {query:s} 
    
    for d in range(1, depth):
        _v = r[d-1]
        s = {}
        for k,v in _v.items():
            for i in v:
                s[i[0]] = wm.most_similar(i[0],topn=topn)
        r[d] = s
#     pprint(r)
    for _,v in r.items():
        for _,k in v.items():
            all_r += k 
#     print(all_r)
    all_r = sorted(all_r,key=lambda x:x[1],reverse=True)
    all_r = set([i[0] for i in all_r])
    pp(all_r)
    return all_r
r = search('表示') 

{'@Benjamin',
 'LiveMint',
 'Sangaku',
 'Technobuffalo',
 'swot',
 '一再强调',
 '一再表示',
 '一口咬定',
 '一方面',
 '三缄其口',
 '不久前',
 '不仅',
 '不仅如此',
 '不懂',
 '不是',
 '不过',
 '与此同时',
 '专访',
 '世称',
 '为什么',
 '为此',
 '举例',
 '之所以',
 '之说',
 '乌雷松',
 '也',
 '也许',
 '了解',
 '了解到',
 '争议',
 '争辩',
 '事后',
 '事实上',
 '于李浩',
 '亦称',
 '人士处',
 '人称',
 '什么',
 '介绍',
 '仔细分析',
 '他们',
 '传言',
 '传闻',
 '似乎',
 '但',
 '但是',
 '何参选',
 '你',
 '你问',
 '使得',
 '俗称',
 '值得一提的是',
 '全会也',
 '全会就',
 '关高通',
 '其实',
 '内情',
 '再问',
 '冒称',
 '写到',
 '写道',
 '出面',
 '函复',
 '分析',
 '则',
 '判断',
 '前不久',
 '力挺',
 '单看你',
 '南共在',
 '却说',
 '原因',
 '原是',
 '去找',
 '又称',
 '反复强调',
 '反问',
 '反驳',
 '反驳者',
 '发声明',
 '发表声明',
 '发表谈话',
 '另一方面',
 '只不过',
 '叫作',
 '叫做',
 '可是',
 '可能',
 '叹道',
 '合称',
 '同时',
 '向国港',
 '向宝沃',
 '向宝能',
 '向延足',
 '向旺成',
 '向深铁',
 '向科融',
 '向雷帝',
 '吗',
 '否定',
 '否认',
 '听其言观其行',
 '听说',
 '告诉',
 '告诉他',
 '呢',
 '和',
 '回复',
 '回应',
 '因为',
 '因此',
 '因而',
 '在发言中',
 '在我看来',
 '在此之前',
 '在讲话中',
 '坚称',
 '坦承',
 '坦言',
 '声称',
 '处理意见',
 '处理结果',
 '复信',
 '如是说',
 '媒体报道',
 '宝能早',
 '实际上',
 '宣布'

# 读取新闻

In [168]:
d = pd.read_csv(file_path)
d['content'] = d['content'].apply(lambda x:str(x)
                                  .replace('\n',' ')
                                  .replace('\\n', ' ')
                                  .replace('\u3000',' '))
sentences = d['content'].tolist()
d.head()

,id,content
0,89617,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,89616,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,89615,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。 至于电...
3,89614,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄
4,89613,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……） @深圳交警微博称：昨日清晨交警...


# 开始处理，使用 ltp 提取实体

In [150]:
from pyltp import NamedEntityRecognizer,Postagger,Segmentor
segmentor = Segmentor()
segmentor.load('/root/files/ltp_data_v3.4.0/cws.model')
postagger = Postagger()
postagger.load('/root/files/ltp_data_v3.4.0/pos.model')
recongnizer = NamedEntityRecognizer()
recongnizer.load('/root/files/ltp_data_v3.4.0/ner.model')


# transpose to {'word':words,'entity':[[start,end,org_name]]}
def ltp_ner(news):
    mapper = {'Ni':'org_name','Nh':'person_name'}
    ret = {'word':None,'entity':None}
    words = segmentor.segment(news)
    postags = postagger.postag(words)
    nertags = s.recognize(words,postags)
    entity = []
    k = 0
    while k < len(words):
        if nertags[k] == 'O' or nertags[k].endswith('Ns'):
            k += 1
        elif nertags[k].startswith('S'):
            entity.append([k,k+1,mapper[nertags[k][2:]]])
            k += 1
        elif nertags[k].startswith('B'):
            start = k
            while k < len(words) and not nertags[k].startswith('E'):k+=1
            entity.append([start,k+1,mapper[nertags[k][2:]]])
            k += 1
        else:
            k += 1
    return {
        'word':list(words),
        'entity':entity
    }

def contains_say_keywords(c):
    return any(True if i in c else False for i in say_represents)

def uniteNER(news):
    ner = ltp_ner(news)
#     ner = nlp.ner(news)[0]
    words = ner['word']
    entity = ner['entity']
    N = []
    # record the entity start and end. k:v = start : end
    entity_start = {}
    for e in entity:
        if e[2] in {'org_name','person_name'}:
            entity_start[e[0]] = e[1]
            N.append([''.join(words[e[0]:e[1]]),e[2]])
    return N, entity_start, words

def extract_points(news):
    points = []
    ners, es, words = uniteNER(news)
    k = 0
    while k < len(words):
        if k in es:
            # entity start, fetch the sentence
            # first add the entity
            _p = [''.join(words[k:es[k]])]
            # second add the sentence
            start = k
            # find the finish signal
            while k < len(words) and words[k] not in end:k+=1
            _p.append(''.join(words[start:k]))
            
            points.append(_p)
        k += 1
    # filter the none points sentences. 
    # currently methods is naive: filter the sentence by keywords which can represent says
    points = list(filter(lambda x: contains_say_keywords(x[1]),points))
    
    return points

In [151]:
news = sentences[101]
pp(news)
p = extract_points(news)
for i in p:
    print(i)

('中国经济网6月23日讯（记者刘瑾）日前，中国机械工业百强企业、汽车三十强企业名单出炉。在由中国机械工业联合会、中国汽车工业协会主办的“中国机械工业百强企业、汽车工业三十强企业信息发布会”上，中国机械工业联合会会长王瑞祥指出，入围企业总体呈现了持续增长态势，2016年总规模、平均规模均创新高，我国机械工业总体上呈现稳中向好的发展态势，主要经济指标出现恢复性增长。 '
 '同时，王瑞祥也表示，机械工业长期积累的结构性矛盾还没有根本性改变，新的发展业态、发展模式远未形成，保持平稳健康发展的基础还不牢固，产业升级、新旧动能转换、实现由大变强任重道远。 '
 '相关数据显示，2016年机械百强入围企业规模是15.4亿元，比上年略高。总规模达到16992亿元，平均规模169.92亿元，两项均创新高。相比前两年的前十强排名，名单基本没有什么变化，只是工程机械行业的徐工、三一和中联重科由于之前市场环境等原因，整体排位后移了一两位，但仍全部在十强之列。但考虑到工程机械行业最近以来的强烈增长态势，明年该行业整体恢复性可能较大。 '
 '中机联执行副会长陈斌表示，机械百强的年换位率2005年曾高达32%，此后逐年下降，2012年降至6%，不过近几年又在上升，2016年升至22%，说明市场动荡，不确定因素增加。 '
 '根据中机联公布的数据，2017年1至5月机械增加值增速好于去年但呈现逐月回落的态势，比如1至5月增速为10.7，比1至4月回落0.2点。此外利润总额也出现了同样的走势。专家分析，这是由于钢铁等原材料价格近期出现较大增幅，使得工业企业成本压力增大。 '
 '“产量同比增长表现抢眼的是工程机械类产品，已经有8个月保持高速增长。”同时，陈斌也表示了担忧，比如价格指数缓慢回升但远低于工业、原材料指数，固定资产投资增速仍较低迷，增速在多年回落之后继续在低位震荡徘徊。他表示，行业经济运行仍面临一定的下行压力，机械行业主要服务的钢铁、煤炭、电力及石油、化工等行业普遍处于产业结构深度调整期，其装备需求短期难以大幅增长。还有经过多年的高速发展，各类机械产品的社会保有量均达到了相当规模，对现役设备的更新改造维护已成为需求中的重要部分，这增加了增量回升的难度。 ')
['中国机械工业联合会', '中国机械工业联合会、中国汽车工业协会主办的“中国机械工业百强企业、汽车工业三十强企业信息发布会”

# 哈工大 ltp 工具

In [162]:
from pyltp import NamedEntityRecognizer,Postagger,Segmentor
segmentor = Segmentor()
segmentor.load('/root/files/ltp_data_v3.4.0/cws.model')
postagger = Postagger()
postagger.load('/root/files/ltp_data_v3.4.0/pos.model')
recongnizer = NamedEntityRecognizer()
recongnizer.load('/root/files/ltp_data_v3.4.0/ner.model')


def ltp_prepare(news):
    words = segmentor.segment(news)
    postags = postagger.postag(words)
    nertags = recongnizer.recognize(words,postags)
    return nertags


def ltp_ner(news):
    mapper = {'Ni':'org_name','Nh':'person_name'}
    ret = {'word':None,'entity':None}
    words = segmentor.segment(news)
    postags = postagger.postag(words)
    nertags = recongnizer.recognize(words,postags)
    entity = []
    k = 0
    while k < len(words):
        if nertags[k] == 'O' or nertags[k].endswith('Ns'):
            k += 1
        elif nertags[k].startswith('S'):
            entity.append([k,k+1,mapper[nertags[k][2:]]])
            k += 1
        elif nertags[k].startswith('B'):
            start = k
            while k < len(words) and not nertags[k].startswith('E'):k+=1
            entity.append([start,k+1,mapper[nertags[k][2:]]])
            k += 1
        else:
            k += 1
    return {
        'word':list(words),
        'entity':entity
    }


def test_ltp_ner(ner):
    words = ner['word']
    entity = ner['entity']
    for i in entity:
        print(words[i[0]:i[1]],i[2])
        
test_ltp_ner(ltp_ner(news))

['刘'] person_name


## stanfordcorenlp 需要自己组合。
***

##hit ltp 采用BIESO 标注体系，可以直接看出 ner 类别，然后手工合并，效果不错,参考
https://pyltp.readthedocs.io/zh_CN/latest/api.html
https://ltp.readthedocs.io/zh_CN/latest/appendix.html#id4
![](image/ner.png)
***
## bosonnlp 有次数限制，每天 ner 500 次，效果还可以
bosonlp最好用。
***
刘奶奶的肿瘤已经扩大并转移至肛门，癌细胞也已扩散，令人惋惜。
bosonnlp 可以识别出刘奶奶，但 ltp 只能识别出刘

In [165]:
from __future__ import print_function, unicode_literals
from bosonnlp import BosonNLP
news = '刘奶奶的肿瘤已经扩大并转移至肛门，癌细胞也已扩散，令人惋惜。'
pp(ltp_prepare(news))
for i in ltp_prepare(news):
    print(i)
# nlp = BosonNLP('cKWUytiR.34676.f5F2YbS_EyX2')
# print(nlp.ner(news))

S-Nh
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O


# 存在的问题

网易财经6月28日讯中国银行行长李礼辉在6月28日举行的2013陆家嘴论坛上表示，建议适时取消贷款利率浮动的下限。适当精简人民币基准利率的档次，提高作为基准利率的使用范围和影响力。他认为，现在人民币基准利率的档次比较多，不管是存款的基准利率还是贷款的基准利率应该适当的精简，提高作为基准利率的使用范围和影响力。在实体经济可以承受的前提下，增强人民币汇率的弹性，扩大外汇市场参与主体，增加交易的品种，人民币与更多的外币直接交易，形成更多的直接的双边汇率。以下是发言实录：李礼辉第二个建议，深化利率市场化的改革，形成内外均衡的人民币价格，充分发挥市场在资源配置中的基础性的作用。下一个阶段，我们将可以适时取消贷款利率浮动的下限，适当精简人民币基准利率的档次，现在人民币基准利率的档次比较多，我们认为不管是存款的基准利率还是贷款的基准利率应该适当的精简，提高作为基准利率的使用范围和影响力。在实体经济可以承受的前提下，增强人民币汇率的弹性，扩大外汇市场参与主体，增加交易的品种，人民币与更多的外币直接交易，形成更多的直接的双边汇率。李礼辉中国银行已经在上海设立了总部，专业化经营人民币交易业务。中国银行业是香港、澳门、台湾、马来西亚等国家和地区的人民币的清算行，中国银行所做的人民币的跨境业务居市场的前列，我们将致力于提升上海作为国际金融中心的定位和作用，致力于促进香港新加坡等地跨境人民币业务的发展，我们希望能够为人民币的国际化做出我们应有的贡献。谢谢！李礼辉第三，离岸在岸的人民币市场协同发展，继续拓展香港人民币离岸市场的广度和深度，发挥香港离岸中心的作用。积极推动其他国际中心发展离岸人民币市场，逐步形成全球的人民币市场体系，同时加强境内和境外市场的衔接，积极探索在境内发展境外人民币业务，特别加快上海国际金融中心的建设，加强上海在跨境人民币产品创新，交易、定价和清算等方面的作用。本文来源：网易财经责任编辑：王晓易0011
## resluts
中国银行:中国银行行长李礼辉在6月28日举行的2013陆家嘴论坛上表示，建议适时取消贷款利率浮动的下限
看到实体应该是李礼辉，但是由于先匹配到了中国银行，所以本剧的主题被列为中国银行。
*** 
可以对生成的结果进行再次处理，查看 keyword 与实体的距离，并令距离最近的实体为最终实体。